In [239]:
import os
import glob
import altair as alt
import pandas as pd
import numpy as np
import warnings
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pathlib import Path
import os
import plotly
from tqdm import tqdm


pd.set_option('display.max_rows', 500)

In [240]:
team = "Chicago Bears"
team_city = "Chicago"
team_underscore = "_".join(team.lower().split(" "))

team_opp = "Miami Dolphins"
team_opp_alt = team_opp
team_city_opp = "Miami"
team_opp_underscore = "_".join(team_opp.lower().split(" "))

team_date = "20221109"
player_date = "20221109"
nfl_date = "20221109"
# bears
# background etc.
primary_color = "#0B162A" 
secondary_color = "#C83803"

opp_primary_color = "#008E97" 
opp_secondary_color = "#FC4C02"

path_base = Path(os.getcwd()).parents[1]

print(path_base)

cols = ["Date", "Opponent", "Result", "W/L", "Div", "Spread", "Total", "Money"]
df_schedule = pd.read_csv(f"{path_base}/data/team_ranks/_nfl_team_{team_underscore}__{team_date}.csv")
df_schedule["Opponent"] = np.where(df_schedule.Location.eq("Away"), "@" + df_schedule.Opponent, df_schedule.Opponent)
record_str = df_schedule[~df_schedule.Div.isnull()].iloc[-1]["W/L"]

df_schedule = df_schedule.fillna("")
df_schedule = df_schedule.head(n=10)[cols]

df_schedule_opp = pd.read_csv(f"{path_base}/data/team_ranks/_nfl_team_{team_opp_underscore}__{team_date}.csv")
df_schedule_opp["Opponent"] = np.where(df_schedule_opp.Location.eq("Away"), "@" + df_schedule_opp.Opponent, df_schedule_opp.Opponent)
record_opp_str = df_schedule_opp[~df_schedule_opp.Div.isnull()].iloc[-1]["W/L"]

df_schedule_opp = df_schedule_opp.fillna("")
df_schedule_opp = df_schedule_opp.head(n=10)[cols]

file_path = f"{path_base}/data/team_ranks/_nfl_player_stat_*{team_date}.csv"
file_paths = glob.glob(os.path.join(file_path))

file_path = f"{path_base}/data/team_ranks/_nfl_stat*{nfl_date}.csv"
file_paths = glob.glob(os.path.join(file_path))
pds = list()

pbar = tqdm(file_paths)
for file_path in pbar:
    pbar.set_description(file_path)
    df = pd.read_csv(file_path)
    pds.append(df)
df = pd.concat(pds)

df["_stat"] = df.url.str.split("/", expand=True)[3]
df["Stat"] = df._stat.str.split("-").str.join(" ").str.title()
df["Stat"] = df.Stat.str.replace("Per Game", "Pg").str.replace("Opponent", "Opp")\
.str.replace("Offensive", "Off")\
.str.replace("Defensive", "Def")\
.str.replace("Touchdowns", "Td")\
.str.replace("Rushing", "Run")\
.str.replace("Passing", "Pass")\
.str.replace("Interceptions", "Int")

df["_team"] = df.Team.str.lower().str.split(" ").str.join("-")

# TODO do a second lookup with keys reversed
stat_matchup_map = {
    "points-per-game":"opponent-points-per-game",
    "points-per-play":"opponent-points-per-play",
    "passing-yards-per-game":"opponent-passing-yards-per-game",
    # sacks-per-game -> defense
    "qb-sacked-per-game":"sacks-per-game",
    "interceptions-per-game":"interceptions-thrown-per-game",
    "1st-half-points-per-game":"opponent-1st-half-points-per-game",
    "2nd-half-points-per-game":"opponent-2nd-half-points-per-game",
    "rushing-yards-per-game":"opponent-rushing-yards-per-game",
    "passing-yards-per-game":"opponent-passing-yards-per-game",
    "rushing-touchdowns-per-game":"opponent-rushing-touchdowns-per-game",
    "passing-touchdowns-per-game":"opponent-passing-touchdowns-per-game",
    "yards-per-game":"opponent-yards-per-game",
    "yards-per-play":"opponent-yards-per-play",
    "yards-per-pass-attempt":"opponent-yards-per-pass-attempt",
    "yards-per-rush-attempt":"opponent-yards-per-rush-attempt",
    "red-zone-scores-per-game":"opponent-red-zone-scores-per-game",
    "red-zone-scoring-pct":"opponent-zone-scoring-pct",
    "penalties-per-game":"opponent-penalties-per-game",
    "offensive-touchdowns-per-game":"opponent-offensive-touchdowns-per-game",
    "defensive-touchdowns-per-game":"opponent-defensive-touchdowns-per-game",
    "average-team-passer-rating":"opponent-average-team-passer-rating",
    "seconds-per-play":"opponent-seconds-per-play",
    "giveaways-per-game":"takeaways-per-game"
}
team_matchup_map = {"chicago":"detroit"}

df = df.drop(columns=["Unnamed: 0"])


/home/jkonovsky/development/notebooks/chicago_bears_nerd


/home/jkonovsky/development/notebooks/chicago_bears_nerd/data/team_ranks/_nfl_stat_opponent_penalties_per_play_20221109.csv: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 226/226 [00:00<00:00, 531.97it/s]


In [241]:
def find_matchup_rank(df, team_matchup_map, stat_matchup_map):
    df["matchup_key"] =  df._team +"-"+ df._stat
    df["matchup_team"] = df._team.map(team_matchup_map).str.title()
    df["matchup_key_lookup"] = df.matchup_team.str.lower() +"-"+ df._stat.map(stat_matchup_map)
    matchup_dict_data = {k:v for k,v in df.set_index("matchup_key")["Rank"].to_dict().items() if not pd.isna(k)}
    df["matchup_rank"] = df.matchup_key_lookup.map(matchup_dict_data)
    
    matchup_dict_data = {k:v for k,v in df.set_index("matchup_key")["2022"].to_dict().items() if not pd.isna(k)}
    df["matchup_value"] = df.matchup_key_lookup.map(matchup_dict_data)
    return df


In [242]:
#team_matchup_map.update({v:k for k,v in team_matchup_map.items()})
stat_matchup_map.update({v:k for k,v in stat_matchup_map.items()})

df = find_matchup_rank(df, team_matchup_map, stat_matchup_map)
 
#df = find_matchup_rank(df, {v:k for k,v in team_matchup_map.items()}, {v:k for k,v in stat_matchup_map.items()})

In [243]:
df["matchup_diff"] = abs(df.matchup_rank - df.Rank)

In [244]:
df[~df.matchup_rank.isna()][["Team", "matchup_team", "Stat", "2022", "Rank", "matchup_rank", "matchup_value", "matchup_diff"]].query("matchup_diff>=20")

,Team,matchup_team,Stat,2022,Rank,matchup_rank,matchup_value,matchup_diff
0,Chicago,Detroit,Run Yards Pg,195.4,1,31.0,148.8,30.0
1,Chicago,Detroit,Yards Per Rush Attempt,5.4,2,28.0,5.0,26.0
25,Chicago,Detroit,Opp Red Zone Scores Pg,2.3,25,3.0,2.5,22.0
25,Chicago,Detroit,Opp Off Td Pg,2.8,26,5.0,2.9,21.0
31,Chicago,Detroit,Opp 1St Half Points Pg,16.2,32,10.0,13.3,22.0
26,Chicago,Detroit,Opp Yards Per Play,5.8,27,6.0,6.0,21.0
25,Chicago,Detroit,Opp Seconds Per Play,29.9,26,4.0,27.2,22.0
29,Chicago,Detroit,Opp Run Yards Pg,147.2,30,10.0,134.1,20.0
31,Chicago,Detroit,Opp Run Td Pg,1.7,32,9.0,1.1,23.0
29,Chicago,Detroit,Sacks Pg,1.4,30,5.0,1.6,25.0


In [27]:
# create a for loop that finds the opponent rank
# maype a function that sets the value of a column
# and creates a column called matchup rank
# should have something like a bad matchup count
# def find_matchup(home_team, away_team, stat):
# def apply find_matchup:=

In [245]:
df[""]

,Rank,Team,2022,Last 3,Last 1,Home,Away,2021,asof,url,_stat,Stat,_team,matchup_key,matchup_team,matchup_key_lookup,matchup_rank,matchup_value,matchup_diff
0,1,Kansas City,30.4,28.0,20.0,24.3,36.5,29.4,2022-11-09,/nfl/stat/points-per-game,points-per-game,Points Pg,kansas-city,kansas-city-points-per-game,NaN,NaN,NaN,NaN,NaN
1,2,Philadelphia,28.1,30.0,29.0,28.5,27.8,25.5,2022-11-09,/nfl/stat/points-per-game,points-per-game,Points Pg,philadelphia,philadelphia-points-per-game,NaN,NaN,NaN,NaN,NaN
2,3,Buffalo,27.5,22.7,17.0,35.3,22.8,29.8,2022-11-09,/nfl/stat/points-per-game,points-per-game,Points Pg,buffalo,buffalo-points-per-game,NaN,NaN,NaN,NaN,NaN
3,4,Seattle,26.8,31.7,31.0,21.5,31.0,23.2,2022-11-09,/nfl/stat/points-per-game,points-per-game,Points Pg,seattle,seattle-points-per-game,NaN,NaN,NaN,NaN,NaN
4,5,Baltimore,26.0,23.3,27.0,25.0,27.0,22.8,2022-11-09,/nfl/stat/points-per-game,points-per-game,Points Pg,baltimore,baltimore-points-per-game,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,28,LA Rams,0.04,0.04,0.05,0.04,0.04,0.04,2022-11-09,/nfl/stat/opponent-penalties-per-play,opponent-penalties-per-play,Opp Penalties Per Play,la-rams,la-rams-opponent-penalties-per-play,NaN,NaN,NaN,NaN,NaN
28,29,Baltimore,0.04,0.03,0.03,0.05,0.03,0.04,2022-11-09,/nfl/stat/opponent-penalties-per-play,opponent-penalties-per-play,Opp Penalties Per Play,baltimore,baltimore-opponent-penalties-per-play,NaN,NaN,NaN,NaN,NaN
29,30,Indianapolis,0.04,0.03,0.03,0.04,0.03,0.05,2022-11-09,/nfl/stat/opponent-penalties-per-play,opponent-penalties-per-play,Opp Penalties Per Play,indianapolis,indianapolis-opponent-penalties-per-play,NaN,NaN,NaN,NaN,NaN
30,31,Miami,0.04,0.04,0.03,0.03,0.04,0.05,2022-11-09,/nfl/stat/opponent-penalties-per-play,opponent-penalties-per-play,Opp Penalties Per Play,miami,miami-opponent-penalties-per-play,NaN,NaN,NaN,NaN,NaN
